In [1]:
# verificar kernel utilizado
import sys
sys.executable

'/home/henrique/Documents/projects/facudade/ciencia-dos-dados/ciencia_de_dados_av1/.venv/bin/python'

In [2]:
# importar bibliotecas 
import pandas as pd
import numpy as np
from config import *
import json

## Organizar path para arquivos do enem 2023

In [3]:
data_dirs = Config.get_path_dir_data()
data_extract = data_dirs["microdados_enem_2023"].list_dir()["microdados_enem_2023_extract"]
for key, value in data_extract.list_dir().items():
    print(f"{key} : {value}")
extract_dict = data_extract.list_dir()

DICIONÁRIO : /home/henrique/Documents/projects/facudade/ciencia-dos-dados/ciencia_de_dados_av1/files/data/microdados_enem_2023/microdados_enem_2023_extract/DICIONÁRIO
PROVAS E GABARITOS : /home/henrique/Documents/projects/facudade/ciencia-dos-dados/ciencia_de_dados_av1/files/data/microdados_enem_2023/microdados_enem_2023_extract/PROVAS E GABARITOS
LEIA-ME E DOCUMENTOS TÉCNICOS : /home/henrique/Documents/projects/facudade/ciencia-dos-dados/ciencia_de_dados_av1/files/data/microdados_enem_2023/microdados_enem_2023_extract/LEIA-ME E DOCUMENTOS TÉCNICOS
DADOS : /home/henrique/Documents/projects/facudade/ciencia-dos-dados/ciencia_de_dados_av1/files/data/microdados_enem_2023/microdados_enem_2023_extract/DADOS
INPUTS : /home/henrique/Documents/projects/facudade/ciencia-dos-dados/ciencia_de_dados_av1/files/data/microdados_enem_2023/microdados_enem_2023_extract/INPUTS


In [4]:
dicionario = extract_dict["DICIONÁRIO"]
PATH_DICIONARIO = dicionario.list_file()["Dicionário_Microdados_Enem_2023"]

## Carregar dicionário como uma tabela

In [7]:
xl_file = pd.ExcelFile(PATH_DICIONARIO)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [8]:
print(dfs)

{'MICRODADOS_ENEM_2023':                    DICIONÁRIO DE VARIÁVEIS - ENEM 2023             Unnamed: 1  \
0                                                  NaN                    NaN   
1                                     NOME DA VARIÁVEL              Descrição   
2                                                  NaN                    NaN   
3                                DADOS DO PARTICIPANTE                    NaN   
4                                         NU_INSCRICAO  Número de inscrição1    
..                                                 ...                    ...   
343  3. Foi considerado treineiro o inscrito que nã...                    NaN   
344  4. As 45 primeiras posições deste campo são re...                    NaN   
345  5. As 45 primeiras posições deste campo são re...                    NaN   
346  6. As 45 primeiras posições deste campo são re...                    NaN   
347  7. As 50 primeiras posições deste campo são re...                    NaN   

  